In [783]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [784]:
min_weight = 5
max_weight = 15
n_nodes = 5 * 10 ** 4
n_edges = 10 ** 6
n_nodes = 5 * 10 ** 2
n_edges = 10 ** 4
n_nodes = 5
n_edges = 10
# n_nodes, n_edges = 50000, 1000000 # <|-- takes 17.5 s to compute the maximum flow in a graph
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [805]:
def modify_weights_graph (Graph, ctr_lists, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(Graph, 0, name="weight")

    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (ctr_lists, new_weights)}),
        name = 'weight'
    )

In [806]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [807]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

C:\Users\val\AppData\Local\Temp\ipykernel_13556\2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(5, 5)

In [808]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[0]

(30.034077055098578,
 {1: 12.671619125804556, 3: 10.194697906276499, 4: 7.167760023017527})

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [809]:
import pulp as pl

In [810]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex.exe'

In [811]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [812]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

5

## Master

In [813]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [814]:
# useful to know the final paths at the end
path2idx = {}
ctr_lists = []
path_count = 0

In [815]:
def add_paths (lpProb, paths, path_count, path2idx):
    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
                ctr_lists.append((path[index], path[index + 1]))
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [816]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 1), path_count, path2idx)

In [817]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

In [818]:
# affichage du MASTER PROBLEM
lpProb

path_generation:
MAXIMIZE
1*X0 + 0
SUBJECT TO
(0, 1): X0 <= 12.6716191258

VARIABLES
X0 <= 15 Continuous

## Dual variables

In [819]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

## Iterative procedure

In [861]:
go_on = True

# while go_on :
for i in range(1):

    duals = get_duals(lpProb)

    modify_weights_graph (Graph_dual, ctr_lists, duals)

    path_q = shortest_path(Graph_dual, source, destination, weight="weight")

    reduced_cost = 1

    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    if reduced_cost >= 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
        lpProb.solve()
    else :
        go_on = False

In [872]:
get_duals(lpProb)[0:8] # le dual n°0 correspond à la 1re contrainte ([0,1] : le chemin direct).
# Comme le dual vaut 1 --> le coût réduit associé vaut 0. C'est donc un chemin qui améliore.
# De plus, c'est le plus court chemin --> ce chemin est indéfiniment ajouté.

[1.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0]

In [869]:
path2idx
# le dual de [(source, destination)] vaut 1 --> c'est le plus court chemin trouvé.
# Il essaie donc d'ajouter indéfiniment ce chemin qui est déjà dans le problème (variable X0)

{0: '[0, 1]',
 1: '[0, 3, 1]',
 2: '[0, 4, 3, 1]',
 3: '[0, 4, 2, 1]',
 4: '[0, 1]',
 5: '[0, 1]',
 6: '[0, 1]',
 7: '[0, 1]',
 8: '[0, 1]',
 9: '[0, 1]',
 10: '[0, 1]'}

In [874]:
print(path_q) # [(0, 1)] : le chemin le plus court
print(reduced_cost) # coût réduit associé au chemin le plus court [(0, 1)]

[0, 1]
0.0


In [871]:
lpProb

path_generation:
MAXIMIZE
1*X0 + 1*X1 + 1*X10 + 1*X2 + 1*X3 + 1*X4 + 1*X5 + 1*X6 + 1*X7 + 1*X8 + 1*X9 + 0
SUBJECT TO
(0, 1): X0 + X10 + X4 + X5 + X6 + X7 + X8 + X9 <= 12.6716191258

(0, 3): X1 <= 10.1946979063

(3, 1): X1 + X2 <= 10.7305856571

(0, 4): X2 + X3 <= 7.16776002302

(4, 3): X2 <= 10.7139611619

(4, 2): X3 <= 12.9970445578

(2, 1): X3 <= 12.6729857951

VARIABLES
X0 <= 15 Continuous
X1 <= 15 Continuous
X10 <= 15 Continuous
X2 <= 15 Continuous
X3 <= 15 Continuous
X4 <= 15 Continuous
X5 <= 15 Continuous
X6 <= 15 Continuous
X7 <= 15 Continuous
X8 <= 15 Continuous
X9 <= 15 Continuous